In [1]:
from requests import Session, Request
from numba import njit
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
from pprint import pprint as pp
%load_ext line_profiler

In [23]:
class ScrapeData:
    def __init__(self):
        self.set_constants()
        self.establish_session()
        self.scrape_init()
        self.scrape_countries()
        self.scrape_regions()
    
    def set_constants(self):
        self.base_url = 'https://www.wine-searcher.com'
        self.headers_ = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
                   "path" : "/regions",
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                   "Accept-Language": "en-US,en;q=0.5",
                   "Accept-Encoding": "gzip, deflate",
                   "DNT": "1",
                   "sec-fetch-dest" : "document",
                   "sec-fetch-mode" : "navigate",
                   "Connection": "close",
                   "Upgrade-Insecure-Requests": "1"}
    
    def establish_session(self):
        self.session_ = Session()
    
    def scrape_countries(self):
        countries_response_ = self.session_.get(f'{self.base_url}/regions', headers=self.headers_).content
        countries_response_ = BeautifulSoup(countries_response_, 'html.parser')
        countries_eps_ = countries_response_.find(string='Top countries:').find_parent("div").next_sibling.next_sibling
        countries_eps_ = [[ctry_.text, ctry_.a["href"]] for ctry_ in countries_eps_ if not str(ctry_)=="\n"]
        self.data_ = []
        for country_, country_endpoint_ in countries_eps_:
            each_country_ = {}
            each_country_[country_] = [{'country_endpoint' : country_endpoint_},
                                       {"regions" : scrape_regions(country_, country_endpoint_)}]
            self.data_.append(each_country_)
        
    
    def scrape_regions(self, country_, country_endpoint_):
        regions_response_ = self.session_.get(f'{self.base_url}/{country_endpoint_}', headers=self.headers_).content
        regions_response_ = BeautifulSoup(regions_response_, 'html.parser')
        regions_eps_ = regions_response_.find(string=f'Sub-regions of {country_}').find_parent("div").next_sibling.next_sibling
        regions_eps_ = [[reg_.text, reg_.a["href"]] for reg_ in regions_eps_.children if not str(reg_)=='\n']
        list_regions_ = []
        for region_, region_endpoint_ in regions_eps_:
            each_region_ = {}
            each_country_[region_] = [{'region_endpoint' : region_endpoint_},
                                       {"sub-regions" : scrape_subregions(region_, region_endpoint_)}]
            list_regions_.append(each_region_)
        
        
        for country_, country_data_ in self.data_.items():
            response_ = self.session_.get(f'{self.base_url}/regions-{country_}', headers=self.headers_).content
            response_ = BeautifulSoup(response_, 'html.parser')
            regions_ = response_.find(string=f'Sub-regions of {country_}').find_parent("div").next_sibling.next_sibling
            regions_ = [[item.text, item.a["href"]] for item in regions_.children if not str(item)=='\n']
            for region_ in regions_:
                country_data_[1]["regions"].append({region_[0] : [
                    {"region_endpoint" : region_[1]},
                    {"sub-regions" : []}
                ]})
    
    def scrape_subregions(self):
        for country_, country_data_ in self.data_.items():
            for region_, region_data_ in country_data_[1]["region"]:
                response_ = self.session_.get(f'{self.base_url}/regions-{country_}', headers=self.headers_).content
                response_ = BeautifulSoup(response_, 'html.parser')
                regions_ = response_.find(string=f'Sub-regions of {country_}').find_parent("div").next_sibling.next_sibling
                regions_ = [[item.text, item.a["href"]] for item in regions_.children if not str(item)=='\n']
                for region_ in regions_:
                    country_data_[1]["regions"].append({region_[0] : [
                        {"region_endpoint" : region_[1]},
                        {"sub-regions" : []}
                    ]})


In [ ]:
sd = ScrapeData()

In [44]:
#data_ = json.dumps(sd.data_, indent=2)
pp(sd.data_)


{'Argentina': [{'country_endpoint': '/regions-argentina'},
               {'regions': [{'Cafayate - Calchaqui Valley': [{'region_endpoint': '/regions-cafayate+-+calchaqui+valley'},
                                                             {'sub-regions': []}]},
                            {'Catamarca': [{'region_endpoint': '/regions-catamarca'},
                                           {'sub-regions': []}]},
                            {'Famatina': [{'region_endpoint': '/regions-famatina'},
                                          {'sub-regions': []}]},
                            {'Fiambala': [{'region_endpoint': '/regions-fiambala'},
                                          {'sub-regions': []}]},
                            {'Jujuy': [{'region_endpoint': '/regions-jujuy'},
                                       {'sub-regions': []}]},
                            {'La Rioja': [{'region_endpoint': '/regions-la+rioja'},
                                          {'sub-regions': []}

In [ ]:
df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [np.nan, 2, 3],
                   "born": [np.nan, 10,
                            np.nan]})
df["born"].fillna(pd.Series([1, 2]), inplace=True)
print(df)